In [1]:
import numpy as np
import pandas as pd

import sys
sys.path.append("../")

from src.object import *
from src.config import *
from src.create import *

In [2]:
df_name = pd.read_csv("../data/csv/NationalNames.csv")
df_name_list = list(df_name[df_name.Gender=='M'].Name.sample(500))

In [3]:
class Formation:
    def __init__(self, name, formation, formation_priority, formation_num):
        self.name = name
        self.formation = formation
        self.formation_priority = formation_priority
        self.formation_num = formation_num
        self.formation_flat = self.__flat_formation()
        self.players = self.__set_players_position()
    
    def __flat_formation(self):
        output = []
        for _, value in self.formation.items():
            output.extend(value)
        output.append("GK")
        return output
    
    def __set_players_position(self):
        output = {}
        for ff in self.formation_flat:
            output[ff] = []
        output["GK"] = []
        return output
    
    def print_formation(self):
        for pos in self.formation_flat:
            player = self.players[pos]
            if pos == "GK":
                player = player[0]
                print(player.name, '  Rate:', player.main_rate, '(', player.main_position, ')')
                print('-'*25)
                print('  DIV ', player.diving, '  REF ', player.reflexes)
                print('  HAN ', player.handling, '  SPE ', player.speed)
                print('  KIC ', player.kicking, '  POS ', player.positioning)
                print()
            else:
                for player_ in player:
                    print(player_.name, '  Rate:', player_.position_all_rate[player_.partification_position], '(', player_.partification_position, ')')
                    print('-'*25)
                    print('  PAC ', player_.pace, '  DRI ', player_.dribbling)
                    print('  SHO ', player_.shooting, '  DEF ', player_.defending)
                    print('  PAS ', player_.passing, '  PHY ', player_.physicality)
                    print()

In [4]:
class Team:
    def __init__(self, name, formation):
        self.name = name
        self.formation = formation
    
    def set_all_players(self, players):
        self.affilation_players = players
        output = pd.DataFrame(columns=ALL_POSITON)
        output_gk = pd.DataFrame(columns=["GK"])

        for i, p in enumerate(self.affilation_players):
            if p.main_position!="GK":
                df = p.position_all_rate
                output = pd.concat([output, pd.DataFrame(np.array(list(df.values())).reshape(1, -1), columns=df.keys(), index=[i])])
            else:
                output_gk = pd.concat([output_gk, pd.DataFrame([p.main_rate], columns=["GK"], index=[i])])
        output["partification"] = 0
        self.affilation_players_all_rate = output
        self.affilation_players_gk_rate = output_gk
    
    def set_starting_players(self):
        for fp in self.formation.formation_priority:
            select_num = self.formation.formation_num[fp]
            df = self.affilation_players_all_rate[self.affilation_players_all_rate.partification==0]
            select_index = df.sort_values(fp, ascending=False).index.values[:select_num]
            self.affilation_players_all_rate.loc[select_index, "partification"] = 1
            
            for index in select_index:
                self.affilation_players[index].partification = 1
                self.affilation_players[index].partification_position = fp
                self.formation.players[fp].append(self.affilation_players[index])
        
        gk_index = self.affilation_players_gk_rate.sort_values("GK", ascending=False).index.values[0]
        self.formation.players["GK"].append(self.affilation_players[gk_index])

In [5]:
name = "3-4-3"
formation = {"ATT":["LW", "ST", "RW"], "MID":["LM", "CM", "RM"], "DEF":["CB"]}
formation_priority = ["ST", "CB", "CM", "LW", "RW", "LM", "RM"]
formation_num = {"ST":1, "CB":3, "CM":2, "LW":1, "RW":1, "LM":1, "RM":1}

_343 = Formation(name=name,
                 formation=formation,
                 formation_priority=formation_priority,
                 formation_num=formation_num)

In [6]:
players = random_create_players(27, 3, 78, 95, df_name_list)
spurs = Team(name="Spurs", formation=_343)

In [7]:
spurs.set_all_players(players)
spurs.set_starting_players()

In [9]:
spurs.formation.print_formation()

Mabel   Rate: 88 ( LW )
-------------------------
  PAC  92   DRI  93
  SHO  55   DEF  49
  PAS  68   PHY  68

Gleen   Rate: 82 ( ST )
-------------------------
  PAC  67   DRI  61
  SHO  82   DEF  53
  PAS  46   PHY  76

Uthman   Rate: 81 ( RW )
-------------------------
  PAC  79   DRI  87
  SHO  61   DEF  65
  PAS  69   PHY  64

Jayvin   Rate: 76 ( LM )
-------------------------
  PAC  84   DRI  71
  SHO  73   DEF  62
  PAS  72   PHY  80

Sisto   Rate: 79 ( CM )
-------------------------
  PAC  62   DRI  64
  SHO  65   DEF  72
  PAS  84   PHY  80

Gilles   Rate: 78 ( CM )
-------------------------
  PAC  65   DRI  56
  SHO  58   DEF  61
  PAS  88   PHY  76

Anders   Rate: 74 ( RM )
-------------------------
  PAC  66   DRI  65
  SHO  83   DEF  50
  PAS  78   PHY  66

Frederico   Rate: 88 ( CB )
-------------------------
  PAC  50   DRI  61
  SHO  46   DEF  89
  PAS  49   PHY  86

Tonny   Rate: 83 ( CB )
-------------------------
  PAC  70   DRI  62
  SHO  57   DEF  80
  PAS  59   PH

In [ ]:
spurs.formation.players["ST"][0]

In [ ]:
pd.DataFrame([70], columns=["GK"], index=[3])